In [42]:
import pandas as pd
import requests
from datetime import datetime

In [43]:
code = requests.get("https://openweathermap.org")
code

<Response [200]>

In [ ]:
lat = -33.9249
lon = 18.4241
API_key = "YOUR_API_KEY"

api_url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric'

response = requests.get(api_url)
response

<Response [200]>

In [ ]:
# Be able to read all the citys available in the database
# Get the API response depending on the choosen City from the Database
# Format the response into a readable dataframe
# read the new weather data back into sql

# Reading the available cities into a Dataframe

In [ ]:
schema = "cities_database"
host = "localhost"
user = "root"
password = "YOUR PASSWORD"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [46]:
cities_from_sql = pd.read_sql("city", con=connection_string)
city_df = cities_from_sql[["city_id", "city_name", "latitude", "longitude", "timezone"]]

## Making a function to get necessary data

In [47]:
def Get_City(cityName):
    return(city_df.loc[city_df["city_name"] == cityName, ["city_name", "latitude", "longitude", "city_id"]])

In [48]:
var = Get_City("Berlin")
var["city_name"][0]

'Berlin'

In [49]:
city_df["city_name"]

0             Berlin
1            Hamburg
2              Paris
3          Stockholm
4              Tokyo
5               Rome
6             Vienna
7          Innsbruck
8          Cape Town
9    Aix-en-Provence
Name: city_name, dtype: object

In [50]:
if(city_df["city_name"].eq("Paris").any()):
        print("This City is in the database.")

This City is in the database.


# Getting the weather Data depending on the city

In [ ]:
def Get_City_Weather(cityName):
    if(cityName not in city_df["city_name"].to_list()):
        print("This City is not in the database.")
    else:
        city_info = Get_City(cityName)
        lat = city_info["latitude"].iloc[0]
        lon = city_info["longitude"].iloc[0]
        API_key = "YOUR OWN API_KEY"

        api_url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric'

        response = requests.get(api_url)
        return(response)


In [52]:
weather_info = Get_City_Weather("Berlin")
weather_info

<Response [200]>

# Make a DataFrame for the Weather

In [70]:
weather_dict = {
    "city_id": [],
    "start_time": [],
    "temperature": [],
    "humidity": [],
    "rain": [],
    "snow": [],
    "wind_speed": [],
    "time_retrieved": []
}

In [54]:
weather_info = Get_City_Weather("Berlin")
weather_info_json = weather_info.json()

Here you can see that one needs to use iloc as the index is also called with this function

In [56]:
Get_City("Hamburg")

,city_name,latitude,longitude,city_id
1,Hamburg,53.55,10.0,2


In [ ]:
for city in city_df["city_name"]:

    weather_info = Get_City_Weather(city)
    weather_info_json = weather_info.json()

    for time_slot in weather_info_json["list"]:

            current_City = Get_City(city)
            weather_dict["city_id"].append(current_City["city_id"].iloc[0])
            weather_dict["start_time"].append(time_slot["dt_txt"])
            weather_dict["temperature"].append(time_slot["main"]["temp"])
            weather_dict["humidity"].append(time_slot["main"]["humidity"])

            try:
                weather_dict["rain"].append(time_slot["snow"]["3h"])
            except KeyError:
                weather_dict["rain"].append(None)

            try:
                weather_dict["snow"].append(time_slot["snow"]["3h"])
            except KeyError:
                weather_dict["snow"].append(None)

            weather_dict["wind_speed"].append(time_slot["wind"]["speed"])
            weather_dict["time_retrieved"].append(datetime.now())
            

Its all the same length

In [75]:
print(len(weather_dict["city_id"]))
print(len(weather_dict["humidity"]))

400
400


In [76]:
weather_df = pd.DataFrame(weather_dict)

# Send the aquired Data back to SQL

In [82]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   city_id         400 non-null    int64         
 1   start_time      400 non-null    datetime64[ns]
 2   temperature     400 non-null    float64       
 3   humidity        400 non-null    int64         
 4   rain            0 non-null      object        
 5   snow            0 non-null      object        
 6   wind_speed      400 non-null    float64       
 7   time_retrieved  400 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(2)
memory usage: 25.1+ KB


Don't forget to convert your columns to the right format for your database

In [83]:
weather_df["start_time"] = pd.to_datetime(weather_df["start_time"])

In [87]:
weather_df.to_sql("weather", 
                  if_exists="append", 
                  con=connection_string, 
                  index=False)

400